# En produktionsmodell för skog i Sverige, baserad på bestånd från riksskogstaxeringens provytor
## A growth simulator for Swedish forests, based on data from the national forest survey.

### File Data:
Author: Carl Vigren
Date: 2024-02-09


In [22]:
# -*- coding: utf-8 -*-
from __future__ import annotations

from math import pi, exp, log, sqrt
from enum import Enum
import warnings


# -------------------------------------------------------------------
# Swedish enums (exact names/values your tests import/use)
# -------------------------------------------------------------------
class RegionSE(Enum):
    NORRA = "North"
    MELLERSTA = "Central"
    SÖDRA = "South"


class VegetationsKod(Enum):
    # only members used in the tests; add others as needed
    ÖRTER_1 = 1
    BLÅBÄR = 13


class MarkfuktighetKod(Enum):
    TORR_ELLER_MYCKET_TORR = 1
    FRISK = 3
    VÅT_ELLER_MYCKET_VÅT = 5


class Trädslag(Enum):
    TALL = "Tall"
    GRAN = "Gran"
    BJÖRK = "Björk"
    BOK = "Bok"
    EK = "Ek"
    ÖV_LÖV = "Öv.löv"


# -------------------------------------------------------------------
# Helpers
# -------------------------------------------------------------------
def _qmd_cm(BA_m2_per_ha: float, stems_per_ha: float) -> float:
    """
    Quadratic mean DBH (cm). BA in m²/ha; stems per ha.
    BA = N * (π * d_cm^2) / 40000 → d_cm = sqrt(BA * 40000 / (π * N))
    """
    if BA_m2_per_ha <= 0.0 or stems_per_ha <= 0.0:
        return 0.0
    return sqrt(BA_m2_per_ha * 40000.0 / (pi * stems_per_ha))


def _safe_sum(iterable):
    total = 0.0
    for v in iterable:
        total += float(v)
    return total


# -------------------------------------------------------------------
# Site object: accepts both English and Swedish arg names used by tests
# -------------------------------------------------------------------
class EkoStandSite:
    def __init__(
        self,
        # English
        latitude: float | None = None,
        altitude: float | None = None,
        vegetation=None,
        soil_moisture=None,
        H100_Spruce: float | None = None,
        region: str | None = None,
        H100_Pine: float | None = None,
        fertilised: bool = False,
        thinned_5y: bool = False,
        thinned: bool = False,
        TAX77: bool = False,
        # Swedish (the tests pass these)
        latitud: float | None = None,
        höjd_möh: float | None = None,
        vegetation_: VegetationsKod | None = None,  # alias (unused but tolerated)
        markfuktighet: MarkfuktighetKod | None = None,
        h100_gran: float | None = None,
        h100_tall: float | None = None,
        gödslad: bool | None = None,
        gallrad_senaste_5år: bool | None = None,
        gallrad: bool | None = None,
        tax77: bool | None = None,
        klimat_zon: str | None = None,
    ):
        # prefer Swedish names when provided; coerce numeric site vars to floats with safe defaults
        if latitud is not None:
            self.latitude = float(latitud)
        elif latitude is not None:
            self.latitude = float(latitude)
        else:
            self.latitude = 0.0

        if höjd_möh is not None:
            self.altitude = float(höjd_möh)
        elif altitude is not None:
            self.altitude = float(altitude)
        else:
            self.altitude = 0.0

        self.fertilised = bool(gödslad) if gödslad is not None else bool(fertilised)
        self.thinned_5y = bool(gallrad_senaste_5år) if gallrad_senaste_5år is not None else bool(thinned_5y)
        self.thinned = bool(gallrad) if gallrad is not None else bool(thinned)
        self.TAX77 = bool(tax77) if tax77 is not None else bool(TAX77)
        self.klimat_zon = klimat_zon

        # region normalization
        if isinstance(region, RegionSE):
            self.region = region.value
        elif isinstance(region, str) and region in ("North", "Central", "South"):
            self.region = region
        elif isinstance(region, str) and region in ("NORRA", "MELLERSTA", "SÖDRA"):
            self.region = {"NORRA": "North", "MELLERSTA": "Central", "SÖDRA": "South"}[region]
        else:
            self.region = RegionSE.SÖDRA.value if region is None else str(region)

        # site index (H100) handling; tests may pass one or both in Swedish
        if h100_gran is not None:
            H100_Spruce = h100_gran
        if h100_tall is not None:
            H100_Pine = h100_tall

        if H100_Spruce is None and H100_Pine is None:
            raise ValueError("At least one of h100_gran or h100_tall must be provided.")

        if H100_Spruce is None:
            self.H100_Pine = H100_Pine
            self.H100_Spruce = self.__Leijon_Pine_to_Spruce(H100_Pine)
        elif H100_Pine is None:
            self.H100_Spruce = H100_Spruce
            self.H100_Pine = self.__Leijon_Spruce_to_Pine(H100_Spruce)
        else:
            self.H100_Spruce = H100_Spruce
            self.H100_Pine = H100_Pine

        # vegetation & field layer flags + vegcode
        veg_in = vegetation
        if isinstance(vegetation, VegetationsKod):
            veg_in = vegetation.value
        self._set_fieldlayer_and_vegcode(veg_in, self.latitude)

        # soil moisture flags
        sm = soil_moisture
        if isinstance(markfuktighet, MarkfuktighetKod):
            sm = markfuktighet.value
        self.DrySoil = (sm == 1)
        self.WetSoil = (sm == 5)

    def _set_fieldlayer_and_vegcode(self, vegetation_code: int | None, latitude: float | None):
        # Set FieldLayer flags
        if vegetation_code in (13, 14):  # bilberry/cowberry
            self.Bilberry_or_Cowberry = True
            self.HerbsGrassesNoFieldLayer = False
        elif vegetation_code in (1, 2, 3, 4, 5, 6, 8, 9) or (vegetation_code == 7 and (latitude or 0) < 60):
            self.Bilberry_or_Cowberry = False
            self.HerbsGrassesNoFieldLayer = True
        else:
            self.Bilberry_or_Cowberry = False
            self.HerbsGrassesNoFieldLayer = False

        # vegcode mapping (as in your original switch)
        mapping = {
            1: 4, 2: 2.5, 3: 2, 4: 3, 5: 2.5, 6: 2, 7: 3, 8: 2.5, 9: 1.5,
            10: -3, 11: -3, 12: 1, 13: 0, 14: -0.5, 15: -3, 16: -5, 17: -0.5, 18: -1,
        }
        # vegetation_code may be None; ensure we only call mapping.get with an int key
        if isinstance(vegetation_code, int):
            self.vegcode = mapping.get(vegetation_code, 0)
        else:
            self.vegcode = 0

    # --- Leijon conversions (unchanged) ---
    @staticmethod
    def __Leijon_Pine_to_Spruce(H100_Pine):
        if H100_Pine < 8 or H100_Pine > 30:
            warnings.warn("SI Pine may be outside underlying material")
        return exp(-0.9596 * log(H100_Pine * 10) + 0.01171 * (H100_Pine * 10) + 7.9209) / 10

    @staticmethod
    def __Leijon_Spruce_to_Pine(H100_Spruce):
        if H100_Spruce < 8 or H100_Spruce > 33:
            warnings.warn("SI Spruce may be outside underlying material.")
        return exp(1.6967 * log(H100_Spruce * 10) - 0.005179 * (H100_Spruce * 10) - 2.5397) / 10


# -------------------------------------------------------------------
# Base classes
# -------------------------------------------------------------------
class EvenAgedStand:
    @staticmethod
    def getQMD(BA, stems):
        return _qmd_cm(BA, stems)

    @staticmethod
    def getMAI(Volume, TotalAge):
        return Volume / TotalAge


class EkoStandPart(EvenAgedStand):
    def __init__(self, ba, stems, age, trädslag: Trädslag):
        self.stand = None
        self.BA = float(ba)
        self.stems = float(stems)
        self.age = float(age)

        self.trädslag = trädslag

        self.QMDOtherSpecies = 0.0
        self.BAOtherSpecies = 0.0
        self.ba_quotient_acute_mortality = 0.0
        self.QMD = self.getQMD(self.BA, self.stems)
        self.HK = 0.0

    def register_stand(self, stand):
        self.stand = stand

    # Alias used in tests’ _snapshot()
    def volume_m3sk_ha(self, BA, QMD, age, stems, HK):
        return self.getVolume(BA=BA, QMD=QMD, age=age, stems=stems, HK=HK)


# -------------------------------------------------------------------
# Species parts (constructors set trädslag; methods contain your formulas)
# -------------------------------------------------------------------
class EkoSpruce(EkoStandPart):
    def __init__(self, ba, stems, age):
        super().__init__(ba, stems, age, Trädslag.GRAN)

    def getMortality(self, increment=5):
        """
        Bengtsson (1981) handwritten note (as in your code).
        Returns: BAQ due to crowding & its QMD; BAQ due to other & its QMD.
        """
        if self.stand.Site.region in ("North", "Central"):
            AKL = min((int(self.age) // 10) + 1, 17)
            crowding = (-0.2748E-02 + 0.4493E-03 * self.BA + 0.2515E-04 * self.BA ** 2) * increment / 100.0
            other = -0.3150E-03 + 0.3337E-01 * AKL  # fraction of BA over the period
        else:
            stems2 = min(self.stems, 2800)
            crowding = (
                0.1235E-01
                + -0.2749E-02 * self.BA
                + 0.8214E-04 * self.BA ** 2
                + 0.2457E-04 * stems2
                + -0.4498E-08 * stems2 ** 2
            ) * increment / 100.0
            other = 0.36 / 100.0  # interpret as percent over the period

        if crowding > 1:
            crowding = 1.0
        elif crowding < 0:
            crowding = 0.0
        return crowding, 0.9 * self.QMD, other, self.QMD

    def getVolume(self, BA=None, QMD=None, age=None, stems=None, HK=None):
        if self.stand is None:
            raise ValueError("Volume calculator cannot be called before part is connected to EkoStand/EkoStandSite.")
        SIdm = float(self.stand.Site.H100_Spruce or 0.0) * 10
        if self.stand.Site.region == "North":
            b1 = -0.065
            b2 = -2.05
            F4age = (1 - exp(b1 * age))
            F4basal_area = (1 - exp(b2 * BA))
            lnVolume = (
                +0.362521E-02 * BA
                + 1.35682 * log(BA)
                - 1.47258 * QMD
                - 0.438770 * F4basal_area
                + 1.46910 * F4age
                - 0.314730 * log(stems)
                + 0.228700 * log(SIdm)
                + 0.118700E-01 * self.stand.Site.thinned
                + 0.254896E-02 * HK
                + 1.970094
            )
            return exp(lnVolume + 0.0388)
        elif self.stand.Site.region == "Central":
            b1 = -0.065
            b2 = -2.05
            F4age = (1 - exp(b1 * age))
            F4basal_area = (1 - exp(b2 * BA))
            lnVolume = (
                +1.28359 * log(BA)
                - 0.380690 * F4basal_area
                + 1.21756 * F4age
                - 0.216690 * log(stems)
                + 0.350370 * log(SIdm)
                + 0.413000E-01 * self.stand.Site.HerbsGrassesNoFieldLayer
                + 0.362100E-01 * self.stand.Site.thinned
                + 0.268645E-02 * HK
                + 0.700490
            )
            return exp(lnVolume + 0.0563)
        else:
            b1 = -0.04
            b2 = -2.05
            F4age = (1 - exp(b1 * age))
            F4basal_area = (1 - exp(b2 * BA))
            lnVolume = (
                +1.22886 * log(BA)
                - 0.349820 * F4basal_area
                + 0.485170 * F4age
                - 0.152050 * log(stems)
                + 0.337640 * log(SIdm)
                + 0.129800E-01 * self.stand.Site.thinned
                + 0.548055E-03 * HK
                + 0.584600
            )
            return exp(lnVolume + 0.0325)

    def getBAI5(self, ba_quotient_chronic_mortality=0.0, ba_quotient_acute_mortality=0.0):
        if self.stand is None:
            raise ValueError("BAI calculator requires EkoStand/EkoStandSite connected.")
        SIdm = float(self.stand.Site.H100_Spruce or 0.0) * 10

        if self.stand.Site.region == "North":
            independent_vars = (
                -0.767477 * ba_quotient_chronic_mortality
                + -0.514297 * ba_quotient_acute_mortality
                + -1.43974 * self.QMD
                + -0.386338E-02 * self.HK
                + 0.204732 * self.stand.Site.fertilised
                + 0.186343 * self.stand.Site.vegcode
                + 0.392021E-01 * self.stand.Site.Bilberry_or_Cowberry
                + -0.807207E-01 * self.stand.Site.DrySoil
                + 0.833252
            )

            if SIdm < 160:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.736655E-02 * self.BA
                        + 0.875788 * log(self.BA)
                        - 0.642060E-04 * self.stems
                        + 0.125396 * log(self.stems)
                        + 0.159356E-02 * self.age
                        - 0.764340 * log(self.age)
                        - 0.594334E-02 * self.BAOtherSpecies
                    )
                else:
                    dependent_vars = (
                        -0.187226E-01 * self.BA
                        + 0.855970 * log(self.BA)
                        + 0.106942E-03 * self.stems
                        + 0.107612 * log(self.stems)
                        + 0.321033E-02 * self.age
                        - 0.737062 * log(self.age)
                        - 0.206053E-01 * self.BAOtherSpecies
                    )
            elif SIdm < 200:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.191493E-01 * self.BA
                        + 0.942389 * log(self.BA)
                        - 0.145476E-03 * self.stems
                        + 0.158511 * log(self.stems)
                        + 0.289628E-02 * self.age
                        - 0.804217 * log(self.age)
                        - 0.125949E-01 * self.BAOtherSpecies
                    )
                else:
                    dependent_vars = (
                        -0.255254E-01 * self.BA
                        + 0.955380 * log(self.BA)
                        - 0.642149E-04 * self.stems
                        + 0.164265 * log(self.stems)
                        + 0.554025E-02 * self.age
                        - 0.866520 * log(self.age)
                        - 0.889755E-02 * self.BAOtherSpecies
                    )
            else:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.210737E-01 * self.BA
                        + 0.932275 * log(self.BA)
                        - 0.572335E-04 * self.stems
                        + 0.152017 * log(self.stems)
                        + 0.342622E-02 * self.age
                        - 0.811183 * log(self.age)
                        - 0.905176E-02 * self.BAOtherSpecies
                    )
                else:
                    dependent_vars = (
                        -0.133941E-01 * self.BA
                        + 0.837783 * log(self.BA)
                        - 0.245946E-03 * self.stems
                        + 0.205142 * log(self.stems)
                        + 0.602419E-02 * self.age
                        - 0.862195 * log(self.age)
                        - 0.135941E-01 * self.BAOtherSpecies
                    )

            self.BAI5 = exp(dependent_vars + independent_vars + 0.0564)

        elif self.stand.Site.region == "Central":
            independent_vars = (
                -1.16597 * ba_quotient_chronic_mortality
                + -0.299327 * ba_quotient_acute_mortality
                + 0.783806E-01 * self.stand.Site.thinned_5y
                + 0.572131E-01 * self.stand.Site.vegcode
                + -0.112938E-01 * self.stand.Site.WetSoil
                + 0.546176E-01 * self.stand.Site.latitude
                + 0.332621E-01 * self.stand.Site.TAX77
            )

            if SIdm < 180:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.802837E-02 * self.BA
                        + 0.751220 * log(self.BA)
                        - 0.800241E-04 * self.stems
                        + 0.239814 * log(self.stems)
                        - 0.148757E-02 * self.age
                        - 0.476534 * log(self.age)
                        - 0.308451E-01 * self.BAOtherSpecies
                        - 4.02484
                    )
                else:
                    dependent_vars = (
                        -0.330623E-01 * self.BA
                        + 1.06539 * log(self.BA)
                        + 0.145290E-03 * self.stems
                        + 0.422450E-01 * log(self.stems)
                        + 0.110998E-01 * self.age
                        - 1.71468 * log(self.age)
                        - 0.236447E-01 * self.BAOtherSpecies
                        + 1.06383
                    )
            elif SIdm < 220:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.211171E-01 * self.BA
                        + 0.837241 * log(self.BA)
                        - 0.800241E-04 * self.stems
                        + 0.239814 * log(self.stems)
                        + 0.492578E-02 * self.age
                        - 0.839650 * log(self.age)
                        - 0.269523E-02 * self.BAOtherSpecies
                        - 2.91926
                    )
                else:
                    dependent_vars = (
                        -0.180419E-01 * self.BA
                        + 0.943986 * log(self.BA)
                        + 0.145290E-03 * self.stems
                        + 0.422450E-01 * log(self.stems)
                        + 0.525585E-02 * self.age
                        - 0.982261 * log(self.age)
                        - 0.786807E-02 * self.BAOtherSpecies
                        - 1.56544
                    )
            elif SIdm < 260:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.263745E-01 * self.BA
                        + 0.915196 * log(self.BA)
                        - 0.800241E-04 * self.stems
                        + 0.239814 * log(self.stems)
                        - 0.384471E-02 * self.age
                        - 0.847753 * log(self.age)
                        - 0.252559E-01 * self.BAOtherSpecies
                        + 2.85518
                    )
                else:
                    dependent_vars = (
                        -0.217674E-01 * self.BA
                        + 0.847682 * log(self.BA)
                        - 0.145290E-03 * self.stems
                        + 0.422450E-01 * log(self.stems)
                        + 0.101626E-01 * self.age
                        - 1.37782 * log(self.age)
                        - 0.268779E-01 * self.BAOtherSpecies
                        + 0.178428
                    )
            else:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.244742E-01 * self.BA
                        + 0.787195 * log(self.BA)
                        - 0.800241E-04 * self.stems
                        + 0.239814 * log(self.stems)
                        + 0.371613E-02 * self.age
                        - 0.561641 * log(self.age)
                        - 0.298097E-01 * self.BAOtherSpecies
                        - 3.17570
                    )
                else:
                    dependent_vars = (
                        -0.239679E-01 * self.BA
                        + 0.924765 * log(self.BA)
                        + 0.145290E-03 * self.stems
                        + 0.422450E-01 * log(self.stems)
                        + 0.631561E-03 * self.age
                        - 0.893401 * log(self.age)
                        - 0.908286E-02 * self.BAOtherSpecies
                        - 1.46143
                    )

            self.BAI5 = exp(dependent_vars + independent_vars + 0.0712)

        else:
            independent_vars = (
                -0.780391 * ba_quotient_chronic_mortality
                + -0.252170 * ba_quotient_acute_mortality
                + -0.318464E-01 * self.stand.Site.thinned_5y
                + 0.778093E-01 * self.stand.Site.fertilised
                + 0.127135E-02 * SIdm
                + 0.262484E-01 * self.stand.Site.vegcode
                + -0.736690E-01 * self.stand.Site.DrySoil
                + -0.269193E-01 * self.stand.Site.latitude
                + -0.959785E-01 * self.stand.Site.TAX77
            )

            if SIdm < 220:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.149200E-01 * self.BA
                        + 0.794859 * log(self.BA)
                        - 0.120956E-03 * self.stems
                        + 0.255053 * log(self.stems)
                        - 0.720252 * log(self.age)
                        - 0.229139E-01 * self.BAOtherSpecies
                        + 1.52732
                    )
                else:
                    dependent_vars = (
                        -0.227763E-01 * self.BA
                        + 0.838105 * log(self.BA)
                        + 0.519813E-03 * self.stems
                        + 0.141232 * log(self.stems)
                        - 0.722723 * log(self.age)
                        - 0.237689E-01 * self.BAOtherSpecies
                        + 1.93218
                    )
            elif SIdm < 260:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.167127E-01 * self.BA
                        + 0.794738 * log(self.BA)
                        - 0.923244E-04 * self.stems
                        + 0.279717 * log(self.stems)
                        - 0.790588 * log(self.age)
                        - 0.187801E-01 * self.BAOtherSpecies
                        + 1.67230
                    )
                else:
                    dependent_vars = (
                        -0.167448E-01 * self.BA
                        + 0.835811 * log(self.BA)
                        - 0.995431E-04 * self.stems
                        + 0.258612 * log(self.stems)
                        - 0.931549 * log(self.age)
                        - 0.167010E-01 * self.BAOtherSpecies
                        + 2.34225
                    )
            elif SIdm < 300:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.221875E-01 * self.BA
                        + 0.832287 * log(self.BA)
                        - 0.110872E-03 * self.stems
                        + 0.271386 * log(self.stems)
                        - 0.735989 * log(self.age)
                        - 0.196143E-01 * self.BAOtherSpecies
                        + 1.50310
                    )
                else:
                    dependent_vars = (
                        -0.203970E-01 * self.BA
                        + 0.836890 * log(self.BA)
                        - 0.755155E-04 * self.stems
                        + 0.248563 * log(self.stems)
                        - 0.716504 * log(self.age)
                        - 0.151436E-01 * self.BAOtherSpecies
                        + 1.50719
                    )
            else:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.243263E-01 * self.BA
                        + 0.902730 * log(self.BA)
                        - 0.706319E-04 * self.stems
                        + 0.198283 * log(self.stems)
                        - 0.713230 * log(self.age)
                        - 0.135840E-01 * self.BAOtherSpecies
                        + 1.71136
                    )
                else:
                    dependent_vars = (
                        -0.218319E-01 * self.BA
                        + 0.855200 * log(self.BA)
                        - 0.176554E-03 * self.stems
                        + 0.269091 * log(self.stems)
                        - 0.765104 * log(self.age)
                        - 0.180257E-01 * self.BAOtherSpecies
                        + 1.62508
                    )

            self.BAI5 = exp(dependent_vars + independent_vars + 0.0737)


class EkoPine(EkoStandPart):
    def __init__(self, ba, stems, age):
        super().__init__(ba, stems, age, Trädslag.TALL)

    def getMortality(self, increment=5):
        if self.stand.Site.region in ("North", "Central"):
            stems2 = min(self.stems, 2700)
            crowding = (
                0.3143E-01
                + -0.6877E-02 * self.BA
                + 0.2056E-03 * self.BA ** 2
                + 0.2684E-04 * stems2
                + -0.5092E-08 * stems2 ** 2
            ) * increment / 100.0
            other = 0.35 / 100.0
        else:
            stems2 = min(self.stems, 4000)
            crowding = (
                -0.6766E-01
                + -0.1283E-02 * self.BA
                + 0.7748E-04 * self.BA ** 2
                + 0.1441E-03 * stems2
                + -0.1839E-07 * stems2 ** 2
            ) * increment / 100.0
            other = 0.38 / 100.0

        if crowding > 1:
            crowding = 1.0
        elif crowding < 0:
            crowding = 0.0
        return crowding, 0.9 * self.QMD, other, self.QMD

    def getVolume(self, BA=None, QMD=None, age=None, stems=None, HK=None):
        if self.stand is None:
            raise ValueError("Volume calculator cannot be called before part is connected to EkoStand/EkoStandSite.")
        SIdm = float(self.stand.Site.H100_Pine or 0.0) * 10

        if self.stand.Site.region == "North":
            b1 = -0.06
            b2 = -2.3
            F4age = (1 - exp(b1 * age))
            F4basal_area = (1 - exp(b2 * BA))
            lnVolume = (
                +1.24296 * log(BA)
                - 0.472530 * F4basal_area
                + 1.05864 * F4age
                - 0.170140 * log(stems)
                + 0.247550 * log(SIdm)
                + 0.213800E-01 * self.stand.Site.thinned
                + 0.295300E-01 * self.stand.Site.thinned_5y
                + 0.510332E-02 * HK
                + 1.08339
            )
            return exp(lnVolume + 0.0275)

        elif self.stand.Site.region == "Central":
            b1 = -0.06
            b2 = -2.2
            F4age = (1 - exp(b1 * age))
            F4basal_area = (1 - exp(b2 * BA))
            lnVolume = (
                +0.778157E-02 * BA
                + 1.14159 * log(BA)
                + 0.927460 * F4age
                - 0.166730 * log(stems)
                + 0.304900 * log(SIdm)
                + 0.270200E-01 * self.stand.Site.thinned
                + 0.292836E-02 * HK
                + 0.910330
            )
            return exp(lnVolume + 0.0273)

        else:
            b1 = -0.075
            b2 = -2.2
            F4age = (1 - exp(b1 * age))
            F4basal_area = (1 - exp(b2 * BA))
            lnVolume = (
                +1.21272 * log(BA)
                - 0.299900 * F4basal_area
                + 1.01970 * F4age
                - 0.172300 * log(stems)
                + 0.369930 * log(SIdm)
                + 1.65136 * log(self.stand.Site.latitude)
                + 0.349200E-01 * log(self.stand.Site.altitude)
                - 0.197100E-01 * self.stand.Site.HerbsGrassesNoFieldLayer
                + 0.229100E-01 * self.stand.Site.thinned
                + 0.526017E-02 * HK
                - 6.46337
            )
            return exp(lnVolume + 0.0260)

    def getBAI5(self, ba_quotient_chronic_mortality=0.0, ba_quotient_acute_mortality=0.0):
        if self.stand is None:
            raise ValueError("BAI calculator requires EkoStand/EkoStandSite connected.")
        SIdm = float(self.stand.Site.H100_Pine or 0.0) * 10

        if self.stand.Site.region == "North":
            independent_vars = (
                -0.598419 * ba_quotient_chronic_mortality
                + -0.486198 * ba_quotient_acute_mortality
                + -0.952624E-02 * self.HK
                + 0.674527E-01 * self.stand.Site.thinned_5y
                + 0.100135 * self.stand.Site.vegcode
                + -0.104076 * self.stand.Site.WetSoil
                + -0.329437E-01 * log(self.stand.Site.altitude)
                + 0.526479E-01 * self.stand.Site.TAX77
                + 0.164446
            )
            if SIdm < 160:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.342051E-01 * self.BA
                        + 0.757840 * log(self.BA)
                        - 0.161442E-03 * self.stems
                        + 0.367048 * log(self.stems)
                        + 0.313386E-02 * self.age
                        - 0.842335 * log(self.age)
                        - 0.157312E-01 * self.BAOtherSpecies
                    )
                else:
                    dependent_vars = (
                        -0.222808E-01 * self.BA
                        + 0.707173 * log(self.BA)
                        - 0.407064E-03 * self.stems
                        + 0.386522 * log(self.stems)
                        + 0.309020E-02 * self.age
                        - 0.840856 * log(self.age)
                        - 0.168721E-01 * self.BAOtherSpecies
                    )
            elif SIdm < 200:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.264194E-01 * self.BA
                        + 0.759517 * log(self.BA)
                        - 0.172838E-03 * self.stems
                        + 0.354319 * log(self.stems)
                        + 0.282339E-02 * self.age
                        - 0.830969 * log(self.age)
                        - 0.920265E-02 * self.BAOtherSpecies
                    )
                else:
                    dependent_vars = (
                        -0.215557E-01 * self.BA
                        + 0.678298 * log(self.BA)
                        - 0.223194E-03 * self.stems
                        + 0.345910 * log(self.stems)
                        + 0.230893E-02 * self.age
                        - 0.759426 * log(self.age)
                        - 0.129081E-01 * self.BAOtherSpecies
                    )
            else:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.242773E-01 * self.BA
                        + 0.743286 * log(self.BA)
                        - 0.127080E-03 * self.stems
                        + 0.328240 * log(self.stems)
                        + 0.203892E-02 * self.age
                        - 0.756105 * log(self.age)
                        - 0.136312E-01 * self.BAOtherSpecies
                    )
                else:
                    dependent_vars = (
                        -0.100435E-01 * self.BA
                        + 0.659451 * log(self.BA)
                        - 0.181913E-03 * self.stems
                        + 0.369130 * log(self.stems)
                        + 0.227817E-02 * self.age
                        - 0.793134 * log(self.age)
                        - 0.817145E-02 * self.BAOtherSpecies
                    )
            self.BAI5 = exp(dependent_vars + independent_vars + 0.0645)

        elif self.stand.Site.region == "Central":
            independent_vars = (
                -0.757422 * ba_quotient_chronic_mortality
                + -0.819721 * ba_quotient_acute_mortality
                + -0.156937E-01 * self.HK
                + 0.657419E-01 * self.stand.Site.fertilised
                + 0.208293E-02 * SIdm
                + 0.393424E-01 * self.stand.Site.vegcode
                + -0.787040E-01 * self.stand.Site.DrySoil
                + 0.952773E-01 * self.stand.Site.TAX77
                - 0.466279
            )
            if SIdm < 180:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.247769E-01 * self.BA
                        + 0.739123 * log(self.BA)
                        - 0.724080E-04 * self.stems
                        + 0.307962 * log(self.stems)
                        + 0.213813E-02 * self.age
                        - 0.730167 * log(self.age)
                        - 0.304936E-02 * self.BAOtherSpecies
                    )
                else:
                    dependent_vars = (
                        -0.454216E-01 * self.BA
                        + 0.967594 * log(self.BA)
                        + 0.134748E-03 * self.stems
                        + 0.106405 * log(self.stems)
                        + 0.322181E-02 * self.age
                        - 0.559074 * log(self.age)
                        - 0.146382E-01 * self.BAOtherSpecies
                    )
            elif SIdm < 220:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.204976E-01 * self.BA
                        + 0.710569 * log(self.BA)
                        - 0.331436E-04 * self.stems
                        + 0.318007 * log(self.stems)
                        + 0.186999E-02 * self.age
                        - 0.732359 * log(self.age)
                        - 0.488064E-02 * self.BAOtherSpecies
                    )
                else:
                    dependent_vars = (
                        +0.144234E-01 * self.BA
                        + 0.304194 * log(self.BA)
                        - 0.111460E-02 * self.stems
                        + 0.628499 * log(self.stems)
                        + 0.545633E-02 * self.age
                        - 0.977317 * log(self.age)
                        - 0.126636E-01 * self.BAOtherSpecies
                    )
            else:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.242132E-01 * self.BA
                        + 0.746931 * log(self.BA)
                        - 0.120517E-03 * self.stems
                        + 0.327216 * log(self.stems)
                        + 0.254795E-02 * self.age
                        - 0.758639 * log(self.age)
                        - 0.978754E-02 * self.BAOtherSpecies
                    )
                else:
                    dependent_vars = (
                        -0.126617E-01 * self.BA
                        + 0.599420 * log(self.BA)
                        - 0.405408E-03 * self.stems
                        + 0.472836 * log(self.stems)
                        + 0.455547E-02 * self.age
                        - 0.895734 * log(self.age)
                        - 0.106365E-01 * self.BAOtherSpecies
                    )
            self.BAI5 = exp(dependent_vars + independent_vars + 0.0507)

        else:
            independent_vars = (
                -1.04202 * ba_quotient_chronic_mortality
                + -0.637943 * ba_quotient_acute_mortality
                + -1.75160 * self.QMD
                + -0.592599E-02 * self.HK  # assuming HKD typo → HK
                + 0.637421E-01 * self.stand.Site.thinned_5y
                + 0.462966E-01 * self.stand.Site.fertilised
                + 0.522489E-01 * self.stand.Site.vegcode
                + -0.702839E-01 * self.stand.Site.DrySoil
                + -0.111568E-01 * self.stand.Site.latitude
                + -0.466973E-01 * self.stand.Site.TAX77
            )
            if SIdm < 160:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.497800E-01 * self.BA
                        + 1.19990 * log(self.BA)
                        + 0.114548E-04 * self.stems
                        + 0.164713 * log(self.stems)
                        - 0.884162E-03 * self.age
                        - 0.564604 * log(self.age)
                        - 0.153879E-01 * self.BAOtherSpecies
                        + 0.579562
                    )
                else:
                    dependent_vars = (
                        -0.302305E-01 * self.BA
                        + 0.938947 * log(self.BA)
                        + 0.563241E-03 * self.stems
                        + 0.148914 * log(self.stems)
                        + 0.419586E-02 * self.age
                        - 1.15586 * log(self.age)
                        - 0.138465E-01 * self.BAOtherSpecies
                        + 2.72773
                    )
            elif SIdm < 200:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.123212E-01 * self.BA
                        + 0.864851 * log(self.BA)
                        - 0.497769E-04 * self.stems
                        + 0.200066 * log(self.stems)
                        + 0.211976E-02 * self.age
                        - 0.821163 * log(self.age)
                        - 0.941390E-02 * self.BAOtherSpecies
                        + 1.59527
                    )
                else:
                    dependent_vars = (
                        -0.216126E-02 * self.BA
                        + 0.938131 * log(self.BA)
                        - 0.169034E-03 * self.stems
                        + 0.621225E-01 * log(self.stems)
                        + 0.305833E-02 * self.age
                        - 1.18279 * log(self.age)
                        - 0.439063E-03 * self.BAOtherSpecies
                        + 3.39954
                    )
            elif SIdm < 240:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.107718E-01 * self.BA
                        + 0.796896 * log(self.BA)
                        - 0.975686E-04 * self.stems
                        + 0.230066 * log(self.stems)
                        - 0.577520E-03 * self.age
                        - 0.570857 * log(self.age)
                        - 0.155230E-01 * self.BAOtherSpecies
                        + 0.784527
                    )
                else:
                    dependent_vars = (
                        -0.632941E-02 * self.BA
                        + 0.767710 * log(self.BA)
                        - 0.173551E-03 * self.stems
                        + 0.173044 * log(self.stems)
                        + 0.163026E-02 * self.age
                        - 0.945376 * log(self.age)
                        - 0.133437E-01 * self.BAOtherSpecies
                        + 2.49514
                    )
            else:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.738511E-02 * self.BA
                        + 0.809028 * log(self.BA)
                        - 0.207393E-03 * self.stems
                        + 0.199179 * log(self.stems)
                        + 0.259619E-03 * self.age
                        - 0.663161 * log(self.age)
                        - 0.142082E-01 * self.BAOtherSpecies
                        + 1.27892
                    )
                else:
                    dependent_vars = (
                        -0.207497E-01 * self.BA
                        + 1.00931 * log(self.BA)
                        - 0.653755E-05 * self.stems
                        + 0.851371E-01 * log(self.stems)
                        - 0.307386E-02 * self.age
                        - 0.635182 * log(self.age)
                        - 0.110970E-01 * self.BAOtherSpecies
                        + 1.57124
                    )
            self.BAI5 = exp(dependent_vars + independent_vars + 0.0636)


class EkoBirch(EkoStandPart):
    def __init__(self, ba, stems, age):
        super().__init__(ba, stems, age, Trädslag.BJÖRK)

    def getMortality(self, increment=5):
        if self.stand.Site.region in ("North", "Central"):
            crowding = (-0.2513E-01 + 0.5489E-02 * self.BA) * increment / 100.0
            other = 0.78 / 100.0
        else:
            crowding = 0.04 * increment / 100.0
            other = 0.46 / 100.0

        if crowding > 1:
            crowding = 1.0
        elif crowding < 0:
            crowding = 0.0
        return crowding, 0.9 * self.QMD, other, self.QMD

    def getVolume(self, BA=None, QMD=None, age=None, stems=None, HK=None):
        if self.stand is None:
            raise ValueError("Volume calculator cannot be called before part is connected to EkoStand/EkoStandSite.")
        SIdm = float(self.stand.Site.H100_Spruce or 0.0) * 10

        if self.stand.Site.region in ("North", "Central"):
            b1 = -0.035
            b2 = -2.05
            F4age = (1 - exp(b1 * age))
            F4basal_area = (1 - exp(b2 * BA))
            lnVolume = (
                +1.26244 * log(BA)
                - 0.459580 * F4basal_area
                + 0.540420 * F4age
                - 0.176040 * log(stems)
                + 0.201360 * log(SIdm)
                - 1.68251 * log(self.stand.Site.latitude)
                - 0.404000E-01 * log(self.stand.Site.altitude)
                + 0.757200E-01 * self.stand.Site.fertilised
                + 0.301200E-01 * self.stand.Site.thinned
                + 0.401844E-02 * HK
                + 8.44862
            )
            return exp(lnVolume + 0.0755)
        else:
            b1 = -0.07
            b2 = -2.1
            F4age = (1 - exp(b1 * age))
            F4basal_area = (1 - exp(b2 * BA))
            lnVolume = (
                -0.786906E-02 * BA
                + 1.35254 * log(BA)
                - 1.30862 * QMD
                - 0.524630 * F4basal_area
                + 1.01779 * F4age
                - 0.254630 * log(stems)
                + 0.204880 * log(SIdm)
                + 2.75025 * log(self.stand.Site.latitude)
                + 0.774000E-01 * self.stand.Site.fertilised
                + 0.434800E-01 * self.stand.Site.thinned
                + 0.250449E-02 * HK
                - 9.38127
            )
            return exp(lnVolume + 0.0595)

    def getBAI5(self, ba_quotient_chronic_mortality=0.0, ba_quotient_acute_mortality=0.0):
        if self.stand is None:
            raise ValueError("BAI calculator requires EkoStand/EkoStandSite connected.")
        SIdm = float(self.stand.Site.H100_Spruce or 0.0) * 10

        if self.stand.Site.region in ("North", "Central"):
            independent_vars = (
                -0.474848 * ba_quotient_chronic_mortality
                + -0.207333 * ba_quotient_acute_mortality
                + -0.202362E-02 * self.HK
                + 0.914442E-01 * self.stand.Site.thinned_5y
                + 0.176843 * self.stand.Site.fertilised
                + 0.256714 * self.stand.Site.vegcode
                + -0.488706E-01 * self.stand.Site.WetSoil
                + -0.139928E-01 * self.stand.Site.latitude
                + -0.462992 * self.stand.Site.altitude
                + 0.189383 * self.stand.Site.TAX77
            )
            if SIdm < 140:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        +0.281210E-02 * self.BA
                        + 0.718062 * log(self.BA)
                        - 0.264120E-03 * self.stems
                        + 0.360947 * log(self.stems)
                        - 0.513560 * log(self.age)
                        - 0.146581E-01 * self.BAOtherSpecies
                        - 0.768510
                    )
                else:
                    dependent_vars = (
                        +0.856585E-01 * self.BA
                        + 0.488507 * log(self.BA)
                        - 0.549010E-03 * self.stems
                        + 0.467588 * log(self.stems)
                        - 0.618645 * log(self.age)
                        - 0.477226E-02 * self.BAOtherSpecies
                        - 0.768510
                    )
            elif SIdm < 180:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        +0.831133E-02 * self.BA
                        + 0.660201 * log(self.BA)
                        - 0.161770E-03 * self.stems
                        + 0.361272 * log(self.stems)
                        - 0.609806 * log(self.age)
                        - 0.133204E-01 * self.BAOtherSpecies
                        - 0.355882
                    )
                else:
                    dependent_vars = (
                        +0.665931E-02 * self.BA
                        + 0.700295 * log(self.BA)
                        - 0.221485E-03 * self.stems
                        + 0.316196 * log(self.stems)
                        - 0.489888 * log(self.age)
                        - 0.246752E-01 * self.BAOtherSpecies
                        - 0.355882
                    )
            elif SIdm < 220:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.371203E-02 * self.BA
                        + 0.835899 * log(self.BA)
                        - 0.141238E-03 * self.stems
                        + 0.221611 * log(self.stems)
                        - 0.732659 * log(self.age)
                        - 0.131446E-01 * self.BAOtherSpecies
                        + 0.891049
                    )
                else:
                    dependent_vars = (
                        -0.134251E-02 * self.BA
                        + 0.838751 * log(self.BA)
                        - 0.237653E-03 * self.stems
                        + 0.192259 * log(self.stems)
                        - 0.707746 * log(self.age)
                        - 0.499067E-02 * self.BAOtherSpecies
                        + 0.891049
                    )
            else:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.281602E-01 * self.BA
                        + 0.800357 * log(self.BA)
                        + 0.673284E-04 * self.stems
                        + 0.205233 * log(self.stems)
                        - 0.631139 * log(self.age)
                        - 0.176494E-01 * self.BAOtherSpecies
                        + 0.731245
                    )
                else:
                    dependent_vars = (
                        -0.177526E-01 * self.BA
                        + 0.814686 * log(self.BA)
                        + 0.781625E-04 * self.stems
                        + 0.183532 * log(self.stems)
                        - 0.593656 * log(self.age)
                        - 0.211444E-01 * self.BAOtherSpecies
                        + 0.731245
                    )
            self.BAI5 = exp(dependent_vars + independent_vars + 0.1642)

        else:
            independent_vars = (
                -0.617367 * ba_quotient_chronic_mortality
                + -0.350920 * ba_quotient_acute_mortality
                + -0.134245E-02 * self.HK
                + 0.277904 * self.stand.Site.fertilised
                + 0.154562 * self.stand.Site.vegcode
                + 0.554711E-01 * self.stand.Site.TAX77
            )
            if SIdm < 220:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.850224E-02 * self.BA
                        + 0.931518 * log(self.BA)
                        - 0.874696E-04 * self.stems
                        + 0.124964 * log(self.stems)
                        - 0.890226E-02 * self.age
                        - 0.498825 * log(self.age)
                        - 0.493910E-02 * self.BAOtherSpecies
                        - 0.135041
                    )
                else:
                    dependent_vars = (
                        +0.144427 * self.BA
                        + 0.332109 * log(self.BA)
                        - 0.457988E-03 * self.stems
                        + 0.474159 * log(self.stems)
                        + 0.922378E-02 * self.age
                        - 1.50315 * log(self.age)
                        - 0.116043E-01 * self.BAOtherSpecies
                        + 1.19213
                    )
            elif SIdm < 260:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        +0.129783E-01 * self.BA
                        + 0.688150 * log(self.BA)
                        - 0.158067E-03 * self.stems
                        + 0.304149 * log(self.stems)
                        + 0.411176E-02 * self.age
                        - 0.864501 * log(self.age)
                        - 0.533730E-02 * self.BAOtherSpecies
                        - 0.135041
                    )
                else:
                    dependent_vars = (
                        -0.235447E-01 * self.BA
                        + 0.962877 * log(self.BA)
                        + 0.103737E-03 * self.stems
                        + 0.186790 * log(self.stems)
                        - 0.127109E-02 * self.age
                        - 1.02854 * log(self.age)
                        - 0.849201E-02 * self.BAOtherSpecies
                        + 1.19213
                    )
            elif SIdm < 300:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.110984E-01 * self.BA
                        + 0.748193 * log(self.BA)
                        - 0.434390E-04 * self.stems
                        + 0.270476 * log(self.stems)
                        + 0.823613E-03 * self.age
                        - 0.718419 * log(self.age)
                        - 0.174522E-01 * self.BAOtherSpecies
                        - 0.135041
                    )
                else:
                    dependent_vars = (
                        -0.438786E-03 * self.BA
                        + 0.818427 * log(self.BA)
                        - 0.304146E-03 * self.stems
                        + 0.241055 * log(self.stems)
                        + 0.106700E-01 * self.age
                        - 1.16385 * log(self.age)
                        - 0.1978220E-01 * self.BAOtherSpecies
                        + 1.19213
                    )
            else:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.204315E-01 * self.BA
                        + 0.792798 * log(self.BA)
                        - 0.179026E-03 * self.stems
                        + 0.316913 * log(self.stems)
                        + 0.262117E-02 * self.age
                        - 0.791796 * log(self.age)
                        - 0.146037E-01 * self.BAOtherSpecies
                        - 0.135041
                    )
                else:
                    dependent_vars = (
                        +0.255898E-02 * self.BA
                        + 0.730671 * log(self.BA)
                        + 0.256307E-04 * self.stems
                        + 0.256131 * log(self.stems)
                        + 0.126785E-01 * self.age
                        - 1.24005 * log(self.age)
                        - 0.341768E-02 * self.BAOtherSpecies
                        + 1.19213
                    )
            self.BAI5 = exp(dependent_vars + independent_vars + 0.1590)


class EkoBroadleaf(EkoStandPart):
    def __init__(self, ba, stems, age):
        super().__init__(ba, stems, age, Trädslag.ÖV_LÖV)

    def getMortality(self, increment=5):
        if self.stand.Site.region in ("North", "Central"):
            crowding = (-0.7277E-02 + -0.2456E-02 * self.BA + 0.1923E-03 * self.BA ** 2) * increment / 100.0
            other = 0.5 / 100.0
        else:
            crowding = 0.04 * increment / 100.0
            other = 0.46 / 100.0

        if crowding > 1:
            crowding = 1.0
        elif crowding < 0:
            crowding = 0.0
        return crowding, 0.9 * self.QMD, other, self.QMD

    def getVolume(self, BA=None, QMD=None, age=None, stems=None, HK=None):
        if self.stand is None:
            raise ValueError("Volume calculator cannot be called before part is connected to EkoStand/EkoStandSite.")
        SIdm = float(self.stand.Site.H100_Spruce or 0.0) * 10

        if self.stand.Site.region in ("North", "Central"):
            b1 = -0.04
            b2 = -2.3
            F4age = (1 - exp(b1 * age))
            F4basal_area = (1 - exp(b2 * BA))
            lnVolume = (
                +1.26649 * log(BA)
                - 0.580030 * F4basal_area
                + 0.486310 * F4age
                - 0.172050 * log(stems)
                + 0.174930 * log(SIdm)
                - 1.51968 * log(self.stand.Site.latitude)
                - 0.368300E-01 * log(self.stand.Site.altitude)
                + 0.547400E-01 * self.stand.Site.thinned
                + 0.417126E-02 * HK
                + 7.79034
            )
            return exp(lnVolume + 0.0853)
        else:
            b1 = -0.075
            b2 = -2.1
            F4age = (1 - exp(b1 * age))
            F4basal_area = (1 - exp(b2 * BA))
            lnVolume = (
                -0.148700E-01 * BA
                + 1.29359 * log(BA)
                - 0.784820 * F4basal_area
                + 1.18741 * F4age
                - 0.135830 * log(stems)
                + 0.219890 * log(SIdm)
                + 2.02656 * log(self.stand.Site.latitude)
                + 0.242500E-01 * self.stand.Site.thinned
                + 0.859600E-01 * self.stand.StandBA
                + 0.509488E-03 * HK
                + 7.50102
            )
            return exp(lnVolume + 0.0671)

    def getBAI5(self, ba_quotient_chronic_mortality=0.0, ba_quotient_acute_mortality=0.0):
        if self.stand is None:
            raise ValueError("BAI calculator requires EkoStand/EkoStandSite connected.")
        SIdm = float(self.stand.Site.H100_Spruce or 0.0) * 10

        if self.stand.Site.region in ("North", "Central"):
            independent_vars = (
                -0.345933 * ba_quotient_chronic_mortality
                + -0.138015 * self.stand.Site.vegcode
                + -0.650878E-01 * self.stand.Site.Bilberry_or_Cowberry
                + -0.175149E-01 * self.stand.Site.latitude
                + -0.570035E-03 * self.stand.Site.altitude
                + 0.151318 * self.stand.Site.TAX77
            )
            if SIdm < 160:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        +0.865166E-01 * self.BA
                        + 0.755603 * log(self.BA)
                        - 0.806548E-03 * self.stems
                        + 0.275974 * log(self.stems)
                        - 0.540881E-02 * self.age
                        - 0.117056 * log(self.age)
                        - 0.187866E-01 * self.BAOtherSpecies
                        - 1.18519
                    )
                else:
                    dependent_vars = (
                        +0.865166E-01 * self.BA
                        + 0.755603 * log(self.BA)
                        - 0.806548E-03 * self.stems
                        + 0.275974 * log(self.stems)
                        - 0.540881E-02 * self.age
                        - 0.117056 * log(self.age)
                        - 0.187866E-01 * self.BAOtherSpecies
                        - 0.952398
                    )
            elif SIdm < 200:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        -0.129773E-01 * self.BA
                        + 0.989525 * log(self.BA)
                        - 0.715363E-04 * self.stems
                        + 0.490676E-01 * log(self.stems)
                        + 0.218728E-02 * self.age
                        - 0.944317 * log(self.age)
                        - 0.143834E-01 * self.BAOtherSpecies
                        + 2.78296
                    )
                else:
                    dependent_vars = (
                        -0.129773E-01 * self.BA
                        + 0.989525 * log(self.BA)
                        - 0.715363E-04 * self.stems
                        + 0.490676E-01 * log(self.stems)
                        + 0.218728E-02 * self.age
                        - 0.944317 * log(self.age)
                        - 0.143834E-01 * self.BAOtherSpecies
                        + 2.87671
                    )
            elif SIdm < 240:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        +0.517826E-01 * self.BA
                        + 0.768565 * log(self.BA)
                        - 0.381320E-03 * self.stems
                        + 0.201267 * log(self.stems)
                        + 0.131078E-02 * self.age
                        - 0.831523 * log(self.age)
                        - 0.122796E-01 * self.BAOtherSpecies
                        + 1.65650
                    )
                else:
                    dependent_vars = (
                        +0.517826E-01 * self.BA
                        + 0.768565 * log(self.BA)
                        - 0.381320E-03 * self.stems
                        + 0.201267 * log(self.stems)
                        + 0.131078E-02 * self.age
                        - 0.831523 * log(self.age)
                        - 0.122796E-01 * self.BAOtherSpecies
                        + 1.59209
                    )
            else:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        +0.243920E-02 * self.BA
                        + 0.857832 * log(self.BA)
                        - 0.949555E-04 * self.stems
                        + 0.192173 * log(self.stems)
                        - 0.292753E-02 * self.age
                        - 0.570009 * log(self.age)
                        - 0.240816E-01 * self.BAOtherSpecies
                        + 0.916942
                    )
                else:
                    dependent_vars = (
                        +0.243920E-02 * self.BA
                        + 0.857832 * log(self.BA)
                        - 0.949555E-04 * self.stems
                        + 0.192173 * log(self.stems)
                        - 0.292753E-02 * self.age
                        - 0.570009 * log(self.age)
                        - 0.240816E-01 * self.BAOtherSpecies
                        + 1.17865
                    )
            self.BAI5 = exp(dependent_vars + independent_vars + 0.1648)

        else:
            independent_vars = (
                -1.20049 * ba_quotient_chronic_mortality
                + -0.367064 * ba_quotient_acute_mortality
                + 0.125048 * self.stand.Site.thinned_5y
                + 0.246684 * self.stand.Site.fertilised
                + 0.141955 * self.stand.Site.vegcode
                + 0.354866E-01 * self.stand.Site.latitude
                + -0.361988E-03 * self.stand.Site.altitude
            )
            if SIdm < 240:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        +0.857153 * log(self.BA)
                        - 0.541853E-04 * self.stems
                        + 0.152684 * log(self.stems)
                        - 0.803085E-02 * self.age
                        - 0.570230 * log(self.age)
                        - 0.100518 * log(self.BAOtherSpecies)
                        - 1.93895
                    )
                else:
                    dependent_vars = (
                        +0.857153 * log(self.BA)
                        - 0.541853E-04 * self.stems
                        + 0.152684 * log(self.stems)
                        - 0.803085E-02 * self.age
                        - 0.570230 * log(self.age)
                        - 0.100518 * log(self.BAOtherSpecies)
                        - 2.01960
                    )
            elif SIdm < 280:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        +0.794405 * log(self.BA)
                        - 0.247009 * self.stems
                        + 0.202344 * log(self.stems)
                        - 0.250423 * self.age
                        - 0.669629 * log(self.age)
                        - 0.101205 * log(self.BAOtherSpecies)
                        - 1.93895
                    )
                else:
                    dependent_vars = (
                        +0.794405 * log(self.BA)
                        - 0.247009 * self.stems
                        + 0.202344 * log(self.stems)
                        - 0.250423 * self.age
                        - 0.669629 * log(self.age)
                        - 0.101205 * log(self.BAOtherSpecies)
                        - 2.01960
                    )
            elif SIdm < 320:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        +0.782374 * log(self.BA)
                        - 0.125111E-03 * self.stems
                        + 0.239626 * log(self.stems)
                        - 0.787146E-03 * self.age
                        - 0.733575 * log(self.age)
                        - 0.823802E-01 * log(self.BAOtherSpecies)
                        - 1.93895
                    )
                else:
                    dependent_vars = (
                        +0.782374 * log(self.BA)
                        - 0.125111E-03 * self.stems
                        + 0.239626 * log(self.stems)
                        - 0.787146E-03 * self.age
                        - 0.733575 * log(self.age)
                        - 0.823802E-01 * log(self.BAOtherSpecies)
                        - 2.01960
                    )
            else:
                if not self.stand.Site.thinned:
                    dependent_vars = (
                        +0.771398 * log(self.BA)
                        + 0.427071E-04 * self.stems
                        + 0.167037 * log(self.stems)
                        - 0.190695E-02 * self.age
                        - 0.587696 * log(self.age)
                        - 0.113489 * log(self.BAOtherSpecies)
                        - 1.93895
                    )
                else:
                    dependent_vars = (
                        +0.771398 * log(self.BA)
                        + 0.427071E-04 * self.stems
                        + 0.167037 * log(self.stems)
                        - 0.190695E-02 * self.age
                        - 0.587696 * log(self.age)
                        - 0.113489 * log(self.BAOtherSpecies)
                        - 2.01960
                    )
            self.BAI5 = exp(dependent_vars + independent_vars + 0.1734)


class EkoBeech(EkoStandPart):
    def __init__(self, ba, stems, age):
        super().__init__(ba, stems, age, Trädslag.BOK)

    def getMortality(self, increment=5):
        if self.stand.Site.region in ("North", "Central"):
            crowding = (-0.7277E-02 + -0.2456E-02 * self.BA + 0.1923E-03 * self.BA ** 2) * increment / 100.0
            other = 0.5 / 100.0
        else:
            crowding = 0.04 * increment / 100.0
            other = 0.46 / 100.0

        if crowding > 1:
            crowding = 1.0
        elif crowding < 0:
            crowding = 0.0
        return crowding, 0.9 * self.QMD, other, self.QMD

    def getVolume(self, BA=None, QMD=None, age=None, stems=None, HK=None):
        if self.stand is None:
            raise ValueError("Volume calculator cannot be called before part is connected to EkoStand/EkoStandSite.")
        SIdm = float(self.stand.Site.H100_Spruce or 0.0) * 10
        b1 = -0.02
        b2 = -2.3
        F4age = (1 - exp(b1 * age))
        F4basal_area = (1 - exp(b2 * BA))
        lnVolume = (
            -0.111600E-01 * BA
            + 1.30527 * log(BA)
            - 0.676190 * F4basal_area
            + 0.490740 * F4age
            - 0.151930 * log(stems)
            - 0.572600E-01 * log(SIdm)
            + 0.628000E-01 * self.stand.Site.thinned
            + 0.203927E-02 * HK
            + 2.85509
        )
        return exp(lnVolume + 0.0392)

    def getBAI5(self, ba_quotient_chronic_mortality=0.0, ba_quotient_acute_mortality=0.0):
        if self.stand is None:
            raise ValueError("BAI calculator requires EkoStand/EkoStandSite connected.")
        SIdm = self.stand.Site.H100_Spruce * 10

        independent_vars = (
            -0.862301 * ba_quotient_acute_mortality
            + 0.162579E-02 * SIdm
            + 0.538943
        )

        if SIdm < 310:
            if not self.stand.Site.thinned:
                dependent_vars = (
                    +0.948126 * log(self.BA)
                    + 0.563620E-01 * log(self.stems)
                    - 0.751665 * log(self.age)
                    - 0.163302E-01 * self.BAOtherSpecies
                )
            else:
                dependent_vars = (
                    +0.948126 * log(self.BA)
                    + 0.563620E-01 * log(self.stems)
                    - 0.751665 * log(self.age)
                    - 0.163302E-01 * self.BAOtherSpecies
                    + 0.887110E-01
                )
        else:
            if not self.stand.Site.thinned:
                dependent_vars = (
                    +0.821914 * log(self.BA)
                    + 0.102770 * log(self.stems)
                    - 0.753735 * log(self.age)
                    - 0.163641E-01 * self.BAOtherSpecies
                )
            else:
                dependent_vars = (
                    +0.821914 * log(self.BA)
                    + 0.102770 * log(self.stems)
                    - 0.753735 * log(self.age)
                    - 0.163641E-01 * self.BAOtherSpecies
                    + 0.887110E-01
                )

        self.BAI5 = exp(dependent_vars + independent_vars + 0.1379)


class EkoOak(EkoStandPart):
    def __init__(self, ba, stems, age):
        super().__init__(ba, stems, age, Trädslag.EK)

    def getMortality(self, increment=5):
        if self.stand.Site.region in ("North", "Central"):
            crowding = (-0.7277E-02 + -0.2456E-02 * self.BA + 0.1923E-03 * self.BA ** 2) * increment / 100.0
            other = 0.5 / 100.0
        else:
            crowding = 0.04 * increment / 100.0
            other = 0.46 / 100.0

        if crowding > 1:
            crowding = 1.0
        elif crowding < 0:
            crowding = 0.0
        return crowding, 0.9 * self.QMD, other, self.QMD

    def getVolume(self, BA=None, QMD=None, age=None, stems=None, HK=None):
        if self.stand is None:
            raise ValueError("Volume calculator cannot be called before part is connected to EkoStand/EkoStandSite.")
        SIdm = float(self.stand.Site.H100_Spruce or 0.0) * 10
        b1 = -0.055
        b2 = -2.3
        F4age = (1 - exp(b1 * age))
        F4basal_area = (1 - exp(b2 * BA))
        lnVolume = (
            -0.106300E-01 * BA
            + 1.27353 * log(BA)
            - 0.463790 * F4basal_area
            + 0.801580 * F4age
            - 0.157080 * log(stems)
            + 0.159030 * log(SIdm)
            + 0.503200E-01 * self.stand.Site.thinned
            + 0.188030E-02 * HK
            + 1.40608
        )
        return exp(lnVolume + 0.0756)

    def getBAI5(self, ba_quotient_chronic_mortality=0.0, ba_quotient_acute_mortality=0.0):
        if self.stand is None:
            raise ValueError("BAI calculator requires EkoStand/EkoStandSite connected.")
        SIdm = self.stand.Site.H100_Spruce * 10

        independent_vars = (
            -0.389169 * ba_quotient_acute_mortality
            - 0.609667
        )
        if SIdm < 280:
            dependent_vars = (
                +0.896599 * log(self.BA)
                + 0.199354 * log(self.stems)
                - 0.842665 * log(self.age)
                - 0.146432E-01 * self.BAOtherSpecies
            )
        elif SIdm < 320:
            dependent_vars = (
                +0.847420 * log(self.BA)
                + 0.144495 * log(self.stems)
                - 0.727278 * log(self.age)
                - 0.222990E-01 * self.BAOtherSpecies
            )
        else:
            dependent_vars = (
                +0.851362 * log(self.BA)
                + 0.128100 * log(self.stems)
                - 0.667346 * log(self.age)
                - 0.199705E-01 * self.BAOtherSpecies
            )
        self.BAI5 = exp(dependent_vars + independent_vars + 0.1618)


# -------------------------------------------------------------------
# Stand (adds _competition_metrics and _assign_current_state_metrics)
# -------------------------------------------------------------------
class EkoStand(EvenAgedStand):
    """
    parts: list[EkoStandPart]
    site:  EkoStandSite
    """
    def __init__(self, parts, site: EkoStandSite):
        if not all(isinstance(p, EkoStandPart) for p in parts):
            raise ValueError("All StandParts must be EkoStandPart objects!")
        self.parts = parts           # Swedish API used in tests
        self.Parts = self.parts      # Back-compat alias
        self.Site = site

        # species-region sanity (as in your original warnings)
        if any(isinstance(p, EkoBeech) for p in self.parts) and self.Site.region not in ("Central", "South"):
            warnings.warn("Setting Beech stand outside of southern Sweden!")
        if any(isinstance(p, EkoOak) for p in self.parts) and self.Site.region not in ("Central", "South"):
            warnings.warn("Setting Oak stand outside of southern Sweden!")

        for p in self.parts:
            p.register_stand(self)

        # initialize all metrics once
        self._assign_current_state_metrics()

    # ------------------------------------------------------------------
    # NEW helper: competition & interaction (HK) on the *current* state
    # ------------------------------------------------------------------
    def _competition_metrics(self) -> None:
        """Compute per-part competition metrics from the current net state."""
        # ensure own QMD is current
        for p in self.parts:
            p.QMD = self.getQMD(p.BA, p.stems)
        # competition from others
        for p in self.parts:
            BA_other = _safe_sum(q.BA for q in self.parts if q is not p)
            N_other = _safe_sum(q.stems for q in self.parts if q is not p)
            p.BAOtherSpecies = BA_other
            p.QMDOtherSpecies = self.getQMD(BA_other, N_other)
            denom = p.QMD if p.QMD > 0 else 1e-9
            p.HK = (p.QMDOtherSpecies / denom) * BA_other

    # ------------------------------------------------------------------
    # NEW helper: fully assign all CURRENT state metrics
    # ------------------------------------------------------------------
    def _assign_current_state_metrics(self) -> None:
        """
        Recompute everything derived from the current net state:
          - per-part: QMD, competition metrics, volume (p.VOL)
          - stand totals: StandBA, StandStems, StandVOL
        """
        self._competition_metrics()
        for p in self.parts:
            p.VOL = p.getVolume(BA=p.BA, QMD=p.QMD, age=p.age, stems=p.stems, HK=p.HK)

        self.StandBA = _safe_sum(p.BA for p in self.parts)
        self.StandStems = _safe_sum(p.stems for p in self.parts)
        self.StandVOL = _safe_sum(p.VOL for p in self.parts)

    # Back-compat alias used by tests’ _snapshot()
    def _refresh_competition_vars(self) -> None:
        self._assign_current_state_metrics()

    # Thinning: constant-QMD removal; refresh metrics afterward
    def thin(self, removals: dict):
        """
        removals: dict keyed by part.trädslag (enum) or species string; value = BA (m²/ha) to remove.
        Stems removed computed at current QMD (constant‑QMD removal).
        """
        for p in self.parts:
            key_variants = (p.trädslag, getattr(p.trädslag, "value", None), str(getattr(p.trädslag, "value", "")))
            BA_out = 0.0
            for k in key_variants:
                if k in removals:
                    BA_out = float(removals[k])
                    break
            BA_out = max(0.0, min(BA_out, p.BA))
            if BA_out > 0.0 and p.QMD > 0:
                stems_out = BA_out / (pi * (p.QMD / 200.0) ** 2)
                p.BA -= BA_out
                p.stems = max(0.0, p.stems - stems_out)
        self._assign_current_state_metrics()

    # Five-year growth step (thesis order-of-ops). Returns dict with N1/BA1/QMD1/VOL1.
    def grow(self, years: int = 5, apply_mortality: bool = True):
        # 0) Start-of-period: initialize competition/interaction and totals
        self._assign_current_state_metrics()

        # Start-of-period snapshot volumes for bookkeeping
        for p in self.parts:
            p.VOL0 = p.getVolume(BA=p.BA, QMD=p.QMD, age=p.age, stems=p.stems, HK=p.HK)

        # 1) gross BAI + 2) natural mortality
        for p in self.parts:
            if apply_mortality:
                BAQ_crowd, QMD_dead_crowd, BAQ_other, QMD_dead_other = p.getMortality(increment=years)
            else:
                BAQ_crowd = BAQ_other = 0.0
                QMD_dead_crowd = 0.9 * (p.QMD or 0.0)
                QMD_dead_other = (p.QMD or 0.0)

            # gross basal area increment using correct mortality quotients as inputs
            p.getBAI5(
                ba_quotient_chronic_mortality=BAQ_crowd,
                ba_quotient_acute_mortality=BAQ_other,
            )

            # End-of-period before removals
            p.BA1 = p.BA + p.BAI5
            p.stems1 = p.stems  # no ingrowth modeled here
            p.age2 = p.age + years
            p.QMD1 = self.getQMD(p.BA1, p.stems1)

            # Natural mortality based on start-of-period BA
            p.BA_Mortality = (BAQ_crowd + BAQ_other) * p.BA
            if p.QMD > 0:
                stems_other = p.BA * BAQ_other / (pi * ((QMD_dead_other / 200.0) ** 2))
                stems_crowd = p.BA * BAQ_crowd / (pi * ((QMD_dead_crowd / 200.0) ** 2))
            else:
                stems_other = stems_crowd = 0.0
            p.stems_Mortality = max(0.0, stems_other + stems_crowd)

        # 5a) competition + volume at end pre‑removal
        for p in self.parts:
            BA_other = _safe_sum(q.BA1 for q in self.parts if q is not p)
            N_other = _safe_sum(q.stems1 for q in self.parts if q is not p)
            QMD_other = self.getQMD(BA_other, N_other)
            p.HK1 = (QMD_other / (p.QMD1 if p.QMD1 > 0 else 1e-9)) * BA_other

        for p in self.parts:
            p.VOL1 = p.getVolume(BA=p.BA1, QMD=p.QMD1, age=p.age2, stems=p.stems1, HK=p.HK1)

        # 5b) after natural mortality (no thinning inside grow(); do thinning between periods)
        for p in self.parts:
            p.BA2 = max(0.0, p.BA1 - p.BA_Mortality)
            p.stems2 = max(0.0, p.stems1 - p.stems_Mortality)
            p.QMD2 = self.getQMD(p.BA2, p.stems2)

        for p in self.parts:
            BA_other = _safe_sum(q.BA2 for q in self.parts if q is not p)
            N_other = _safe_sum(q.stems2 for q in self.parts if q is not p)
            QMD_other = self.getQMD(BA_other, N_other)
            p.HK2 = (QMD_other / (p.QMD2 if p.QMD2 > 0 else 1e-9)) * BA_other

        for p in self.parts:
            p.VOL2 = p.getVolume(BA=p.BA2, QMD=p.QMD2, age=p.age2, stems=p.stems2, HK=p.HK2)

        # 6) Commit net end-of-period state; return shape expected by tests
        period = {}
        for p in self.parts:
            p.gross_volume_increment = p.VOL1 - p.VOL0
            p.BA, p.stems, p.QMD, p.age, p.VOL = p.BA2, p.stems2, p.QMD2, p.age2, p.VOL2

            key = p.trädslag.value  # "Tall", "Gran", "Björk", ...
            period.setdefault(key, []).append(
                {
                    "N1": p.stems,
                    "BA1": p.BA,
                    "QMD1": p.QMD,
                    "VOL1": p.VOL,
                    # optional provenance (ignored by your asserts)
                    "N0": p.stems1,
                    "BA0": p.BA1,
                    "QMD0": p.QMD1,
                    "VOL0": p.VOL1,
                }
            )

        # final housekeeping for the new state
        self._assign_current_state_metrics()
        return period

    # legacy wrapper used by some old calls
    def grow5(self, mortality=True):
        return self.grow(years=5, apply_mortality=mortality)


In [23]:

def install_eko_compat():
    """
    Make total-stand metrics always available and safe:
      • EkoStand.StandBA and EkoStand.StandStems become lazy properties with setters.
      • QMDs are precomputed (if needed) before the first volume assignment pass.
    Call this once AFTER your Eko* classes are defined and BEFORE constructing any EkoStand.
    """
    g = globals()
    if 'EkoStand' not in g:
        raise RuntimeError("Define EkoStand/EkoStandSite and species classes first.")

    EkoStand = g['EkoStand']

    if getattr(EkoStand, "_eko_forward_safe", False):
        return  # already installed

    # ---- Lazy totals: work even before constructor sets them explicitly
    def _parts(self):
        return getattr(self, 'Parts', getattr(self, 'parts', [])) or []

    def _get_StandBA(self):
        # Use cached value if later set by the model; otherwise compute on the fly
        if hasattr(self, "_StandBA"):
            return self._StandBA
        return float(sum((getattr(p, "BA", 0.0) or 0.0) for p in _parts(self)))

    def _set_StandBA(self, v):
        self._StandBA = float(v)

    def _get_StandStems(self):
        if hasattr(self, "_StandStems"):
            return self._StandStems
        return float(sum((getattr(p, "stems", 0.0) or 0.0) for p in _parts(self)))

    def _set_StandStems(self, v):
        self._StandStems = float(v)

    # Attach as data-descriptor properties (safe to assign later)
    EkoStand.StandBA = property(_get_StandBA, _set_StandBA, doc="Total basal area (m²/ha), lazy-safe.")
    EkoStand.StandStems = property(_get_StandStems, _set_StandStems, doc="Total stems (/ha), lazy-safe.")

    # ---- Guard the first metrics pass: ensure QMD exists before any volume call
    # If your class already does this, the wrapper is a no-op overhead.
    orig = getattr(EkoStand, "_assign_current_state_metrics", None)
    if callable(orig):
        def _assign_current_state_metrics_wrapped(self, *a, **kw):
            parts = _parts(self)
            # Ensure QMD is available before any formula may try to use it
            for p in parts:
                if (getattr(p, "QMD", None) in (None, 0)) and (getattr(p, "BA", 0) > 0) and (getattr(p, "stems", 0) > 0):
                    try:
                        p.QMD = self.getQMD(p.BA, p.stems)
                    except Exception:
                        pass
            return orig(self, *a, **kw)
        EkoStand._assign_current_state_metrics = _assign_current_state_metrics_wrapped

    EkoStand._eko_forward_safe = True

# Call once after your EKO classes are defined:
install_eko_compat()

In [24]:
import os
import json
import math
import pandas as pd

# ----------------------------- #
# Helpers
# ----------------------------- #

def _to_num(x):
    try:
        if pd.isna(x): 
            return None
    except Exception:
        pass
    try:
        return float(x)
    except Exception:
        try:
            # Some exports use comma decimal separator
            s = str(x).replace(",", ".")
            return float(s)
        except Exception:
            return None

def _to_str(x):
    s = str(x)
    return None if s == 'nan' else s

# ----------------------------- #
# Sheet parsers
# ----------------------------- #

def _parse_site_variables(xlsx_path: str) -> dict:
    """
    Reads the 'Site Variables' sheet and extracts:
      - latitude, altitude, region, soil_moisture_code (1=dry, 3=mesic, 5=wet),
        vegetation_code (1 herbs/grass, 13 bilberry, 14 cowberry ... minimal map here),
      - H100 per species from 'Ståndortsindex, dm' (converted to meters: dm/10).
    """
    df = pd.read_excel(xlsx_path, sheet_name="Site Variables", header=None)

    def find(label):
        for i in range(df.shape[0]):
            for j in range(df.shape[1]):
                if str(df.iat[i, j]).strip().lower() == label.lower():
                    return (i, j)
        return None

    # Geographic
    lat_pos = find('Latitud')
    alt_pos = find('Altitud')
    omr_pos = find('Område')
    lat = _to_num(df.iat[lat_pos[0]+1, lat_pos[1]]) if lat_pos else None
    alt = _to_num(df.iat[alt_pos[0]+1, alt_pos[1]]) if alt_pos else None
    region_raw = _to_str(df.iat[omr_pos[0]+1, omr_pos[1]]) if omr_pos else None
    region_map = {
        'Syd': 'South', 'Södra': 'South', 'SÖDRA': 'South', 'Södra Sverige': 'South',
        'Mellan': 'Central', 'Centrala': 'Central', 'Central': 'Central',
        'Nord': 'North', 'Norra': 'North'
    }
    region = region_map.get(region_raw, None)

    # Soil moisture (very simple mapping)
    torr_pos = find('Torr')
    vat_pos = find('Våt')
    soil_code = 3
    if torr_pos and _to_str(df.iat[torr_pos[0]+1, torr_pos[1]]) in ('Ja', 'True', '1'):
        soil_code = 1
    if vat_pos and _to_str(df.iat[vat_pos[0]+1, vat_pos[1]]) in ('Ja', 'True', '1'):
        soil_code = 5

    # Vegetation (very simple mapping)
    ort_pos = find('Ört/gräs')
    blabar_pos = find('Blåbär/lingon')
    veg_code = None
    if ort_pos and _to_str(df.iat[ort_pos[0]+1, ort_pos[1]]) in ('Ja', 'True', '1'):
        veg_code = 1
    if blabar_pos and _to_str(df.iat[blabar_pos[0]+1, blabar_pos[1]]) in ('Ja', 'True', '1'):
        veg_code = 13

    # H100 site indices (dm → m)
    si_label = find('Ståndortsindex, dm')
    H100 = {}
    if si_label:
        species_row = df.iloc[si_label[0]+1].tolist()
        values_row = df.iloc[si_label[0]+2].tolist()
        for name, val in zip(species_row, values_row):
            if pd.isna(name) or pd.isna(val):
                continue
            H100[str(name)] = _to_num(val) / 10.0 if _to_num(val) is not None else None

    return {
        'latitude': lat,
        'altitude_m': alt,
        'region': region,
        'soil_moisture_code': soil_code,
        'vegetation_code': veg_code,
        'H100': H100
    }

def _parse_general_sheet(xlsx_path: str) -> list[dict]:
    """
    Reads 'General' and returns a list of events of the form:
      {'period': int, 'type': 'Start'|'Tillväxt'|'Gallring',
       'species': { 'Tall': {...}, 'Gran': {...}, ... }}

    Under each species we include:
      - ages: total_age, bh_age, h_top_m
      - 'after' state (Stamantal st/ha, Grundyta m2/ha, Dg cm, Volym m3sk/ha)
      - extraction (Uttag/självgallring), growth (Årlig tillväxt), and mortality flags
    """
    df = pd.read_excel(xlsx_path, sheet_name="General", header=None)
    events_idx = [i for i in range(len(df)) if df.iloc[i, 1] in ('Start', 'Tillväxt', 'Gallring')]
    species_order = ['Tall', 'Gran', 'Björk', 'Bok', 'Ek', 'Öv.löv']

    events = []
    for ei, i in enumerate(events_idx):
        typ = df.iloc[i, 1]
        period = df.iloc[i, 0]
        try:
            period = int(period)
        except Exception:
            period = ei if typ == 'Start' else (events[-1]['period'] + 1 if events else 0)

        # species rows for a block can start 1 row above the event label in some exports (e.g., 'Tall' above 'Start')
        next_boundary = next((idx for idx in events_idx if idx > i), len(df))
        species_block = {}
        k = max(0, i - 1)
        while k < next_boundary:
            sp = df.iloc[k, 2]
            if sp in species_order:
                species_block[sp] = {
                    'total_age': _to_num(df.iloc[k, 3]),
                    'bh_age': _to_num(df.iloc[k, 4]),
                    'h_top_m': _to_num(df.iloc[k, 5]),
                    'after': {
                        'N_stems_ha': _to_num(df.iloc[k, 6]),
                        'BA_m2_ha': _to_num(df.iloc[k, 7]),
                        'QMD_cm': _to_num(df.iloc[k, 8]),
                        'VOL_m3sk_ha': _to_num(df.iloc[k, 9]),
                    },
                    'extraction': {
                        'N_stems_ha': _to_num(df.iloc[k, 11]),
                        'BA_m2_ha': _to_num(df.iloc[k, 12]),
                        'QMD_cm': _to_num(df.iloc[k, 13]),
                        'VOL_m3sk_ha': _to_num(df.iloc[k, 14]),
                    },
                    'growth': {
                        'lopande_m3sk_ha': _to_num(df.iloc[k, 16]),
                        'medel_m3sk_ha': _to_num(df.iloc[k, 17]),
                    },
                    'mortality': {
                        'slow_BA_frac': _to_num(df.iloc[k, 18]),
                        'fast_BA_frac': _to_num(df.iloc[k, 19]),
                    },
                    'flags': {
                        'nygallara': _to_str(df.iloc[k, 20]),
                        'gallrad_nagongang': _to_str(df.iloc[k, 21]),
                        'gallringshistorik': _to_str(df.iloc[k, 22]),
                    },
                }
            k += 1

        events.append({'period': period, 'type': typ, 'species': species_block})

    return events

def excel_to_json(xlsx_path: str) -> dict:
    """
    High-level: parse the Excel and return a single, tidy structure.
    """
    return {
        'source_file': os.path.basename(xlsx_path),
        'site': _parse_site_variables(xlsx_path),
        'events': _parse_general_sheet(xlsx_path),
    }

test1 = excel_to_json("./Output2.xlsx")
test2 = excel_to_json("./Output3.xlsx")
test3 = excel_to_json("./Output4.xlsx")


In [25]:
# ----------------------------- #
# Model replay utilities (use your notebook's EKO classes)
# ----------------------------- #

SWE_TO_CLASSNAME = {
    'Tall': 'EkoPine',
    'Gran': 'EkoSpruce',
    'Björk': 'EkoBirch',
    'Bok': 'EkoBeech',
    'Ek': 'EkoOak',
    'Öv.löv': 'EkoBroadleaf',
}
ENG_FROM_CLASS = {
    'EkoPine': 'Pine',
    'EkoSpruce': 'Spruce',
    'EkoBirch': 'Birch',
    'EkoBeech': 'Beech',
    'EkoOak': 'Oak',
    'EkoBroadleaf': 'Broadleaf',
}

def _build_stand_from_json(json_obj: dict):
    """
    Build EkoStandSite and EkoStand from the JSON.
    Assumes the following names exist in globals(): 
      EkoStandSite, EkoStand, EkoPine, EkoSpruce, EkoBirch, EkoBeech, EkoOak, EkoBroadleaf
    """
    g = globals()
    required = ['EkoStandSite', 'EkoStand'] + list(SWE_TO_CLASSNAME.values())
    missing = [r for r in required if r not in g]
    if missing:
        raise RuntimeError(f"These model classes must be defined in the notebook before running: {missing}")

    site = json_obj['site'] or {}
    H100 = site.get('H100', {})

    # Try keywords to be robust to the __init__ signature in your notebook
    site_kwargs = dict(
        latitude       = site.get('latitude', 56.0),
        altitude       = site.get('altitude_m', 100.0),
        vegetation     = site.get('vegetation_code', 13),
        soil_moisture  = site.get('soil_moisture_code', 3),
        H100_Pine      = H100.get('Tall', None),
        H100_Spruce    = H100.get('Gran', None),
        region         = site.get('region', 'South'),
        fertilised     = False,
        thinned_5y     = False,
        thinned        = False,
        TAX77          = False,
    )

    StandSite = g['EkoStandSite'](**site_kwargs)

    # Build initial parts from period 0 'Start'
    start_event = next(e for e in json_obj['events'] if e['type'] == 'Start')
    parts = []
    for swe_name, cls_name in SWE_TO_CLASSNAME.items():
        if swe_name in start_event['species']:
            rec = start_event['species'][swe_name]
            BA   = (rec.get('after') or {}).get('BA_m2_ha') or 0.0
            N    = (rec.get('after') or {}).get('N_stems_ha') or 0.0
            age  = rec.get('total_age') or 0.0
            if BA > 0 or N > 0:
                parts.append(g[cls_name](BA, N, age))

    Stand = g['EkoStand'](parts, StandSite)
    return StandSite, Stand

def _snapshot(Stand, label=None):
    """
    Capture current per-species values using the model's own volume function.
    Returns:
      {'event': label, 'species': {'Pine': {N, BA, QMD, VOL, age}, ...}}
    """
    # QMD for current state
    for p in Stand.Parts:
        p.QMD = Stand.getQMD(p.BA, p.stems)

    # Competition for current state
    Stand._refresh_competition_vars()

    snap = {'event': label, 'species': {}}
    for p in Stand.Parts:
        vol = p.getVolume(BA=p.BA, QMD=p.QMD, age=p.age, stems=p.stems, HK=p.HK)
        cls = p.__class__.__name__
        eng = ENG_FROM_CLASS.get(cls, cls)
        snap['species'][eng] = dict(N=p.stems, BA=p.BA, QMD=p.QMD, VOL=vol, age=p.age)
    return snap

def _thin_to_match_after_state(Stand, event_species_block: dict):
    """
    Instantaneously set BA and N to the 'after' values for this Gallring event.
    Age remains unchanged. Then recompute QMD, competition, volume.
    """
    # Set target BA and N per species
    for p in Stand.Parts:
        eng = ENG_FROM_CLASS.get(p.__class__.__name__, None)
        # map English class → Swedish key in JSON
        swe_key = next((k for k, v in SWE_TO_CLASSNAME.items() if v == p.__class__.__name__), None)
        if swe_key and swe_key in event_species_block:
            after = (event_species_block[swe_key] or {}).get('after') or {}
            if after.get('BA_m2_ha') is not None:
                p.BA = float(after['BA_m2_ha'])
            if after.get('N_stems_ha') is not None:
                p.stems = float(after['N_stems_ha'])
            p.QMD = Stand.getQMD(p.BA, p.stems)

    # Refresh competition + volumes
    Stand._refresh_competition_vars()
    for p in Stand.Parts:
        p.QMD = Stand.getQMD(p.BA, p.stems)
        _ = p.getVolume(BA=p.BA, QMD=p.QMD, age=p.age, stems=p.stems, HK=p.HK)

    # Mark as thinned (so growth equations that include 'thinned' effects can notice)
    if hasattr(Stand, 'Site'):
        setattr(Stand.Site, 'thinned', True)
        setattr(Stand.Site, 'thinned_5y', True)

def run_management_from_json(json_obj: dict) -> list[dict]:
    """
    Replays the entire sequence encoded in json_obj['events'] against the model.
    Returns snapshots (list of dicts), one per event, in the same order as the Excel blocks.
    """
    _, Stand = _build_stand_from_json(json_obj)

    snapshots = []
    # First snapshot corresponds to 'Start' after-state (already set by initial parts)
    snapshots.append(_snapshot(Stand, label="Start"))

    # Process subsequent events
    for ev in json_obj['events'][1:]:
        if ev['type'] == 'Tillväxt':
            # 5-year growth with mortality
            Stand.grow5(mortality=True)
            if hasattr(Stand.Site, 'thinned_5y'):
                Stand.Site.thinned_5y = False  # thinning was >5y ago for the next interval
            snapshots.append(_snapshot(Stand, label=f"Tillväxt {ev['period']}"))
        elif ev['type'] == 'Gallring':
            # Instant thinning to the event's 'after' values (no age change)
            _thin_to_match_after_state(Stand, ev['species'])
            snapshots.append(_snapshot(Stand, label=f"Gallring {ev['period']}"))
        else:
            # Unknown or other event labels → just snapshot
            snapshots.append(_snapshot(Stand, label=f"{ev['type']} {ev['period']}"))

    return snapshots

def expected_from_json(json_obj: dict) -> list[dict]:
    """
    Pull the Excel 'after' values for each event/species: this is our external model target.
    Returned as a list aligned to events: 
      [{'event_type': 'Start', 'period': 0, 'species': {'Tall': {N, BA, QMD, VOL, age}, ...}}, ...]
    """
    out = []
    for ev in json_obj['events']:
        spec = {}
        for swe, rec in (ev.get('species') or {}).items():
            after = rec.get('after') or {}
            spec[swe] = {
                'N': after.get('N_stems_ha'),
                'BA': after.get('BA_m2_ha'),
                'QMD': after.get('QMD_cm'),
                'VOL': after.get('VOL_m3sk_ha'),
                'age': rec.get('total_age'),
            }
        out.append({'event_type': ev['type'], 'period': ev['period'], 'species': spec})
    return out

snaps1 = run_management_from_json(test1)
snaps2 = run_management_from_json(test2)
snaps3 = run_management_from_json(test3)


C:\Users\csvi0001\AppData\Local\Temp\ipykernel_43192\1397950120.py:1677: UserWarning: Setting Beech stand outside of southern Sweden!
  warnings.warn("Setting Beech stand outside of southern Sweden!")
C:\Users\csvi0001\AppData\Local\Temp\ipykernel_43192\1397950120.py:1679: UserWarning: Setting Oak stand outside of southern Sweden!
  warnings.warn("Setting Oak stand outside of southern Sweden!")


ValueError: math domain error

In [ ]:
import ipytest 
ipytest.autoconfig()

In [ ]:
# If you use ipytest in the notebook:
%%ipytest -q

import pytest

ENG_TO_SWE = {'Pine': 'Tall','Spruce': 'Gran','Birch': 'Björk','Beech': 'Bok','Oak': 'Ek','Broadleaf': 'Öv.löv'}

def _align_snapshot_to_swe(snap: dict) -> dict:
    """
    Model snapshot uses English keys ('Pine', 'Spruce', ...).
    Convert them to Swedish keys so we can compare directly to the Excel JSON ('Tall', 'Gran', ...).
    """
    out = {}
    for eng, vals in (snap.get('species') or {}).items():
        swe = ENG_TO_SWE.get(eng, eng)
        out[swe] = vals
    return out

@pytest.mark.parametrize("xls_path", [
    "/mnt/data/Output3.xlsx",
    "/mnt/data/output4.xlsx",
    "/mnt/data/Output2.xlsx",
])
def test_model_matches_excel_within_tolerance(xls_path):
    data = excel_to_json(xls_path)
    expected = expected_from_json(data)
    model_snaps = run_management_from_json(data)

    assert len(model_snaps) == len(expected), \
        f"Different number of events: model {len(model_snaps)} vs expected {len(expected)}"

    # Compare per event, per species
    for i, (exp_ev, snap) in enumerate(zip(expected, model_snaps)):
        snap_swe = _align_snapshot_to_swe(snap)
        for swe, rec in (exp_ev.get('species') or {}).items():
            # If a species is not present in the event, skip
            if rec['BA'] is None and rec['N'] is None:
                continue
            assert swe in snap_swe, f"Species {swe} missing in model snapshot for event index {i}"
            got = snap_swe[swe]

            # Absolute tolerances (tune if you need tighter/looser)
            if rec['N'] is not None:
                assert got['N'] == pytest.approx(rec['N'], abs=25)
            if rec['BA'] is not None:
                assert got['BA'] == pytest.approx(rec['BA'], abs=0.8)
            if rec['QMD'] is not None:
                assert got['QMD'] == pytest.approx(rec['QMD'], abs=0.8)
            if rec['VOL'] is not None:
                assert got['VOL'] == pytest.approx(rec['VOL'], abs=8)


UsageError: Line magic function `%%ipytest` not found.
